In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random

In [6]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import warnings
# warnings.filterwarnings('ignore')
# import seaborn as sns
# import sqlite3
# import mysql.connector as cnt
# import plotly.express as px 
# import requests
# import os



opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://autoplius.lt/skelbimai/naudoti-automobiliai?page_nr=149'
driver.get(url)
time.sleep(5)

# page = 1
page = 149
while page < 201:
# while page < 3:
    source = driver.page_source

    bs = BeautifulSoup(source, 'html.parser')
    # print(bs)
    auto_nuorodos = []


    auto_links = bs.find('div', {'class':'auto-lists lt'}).find_all('a')
    for link in auto_links:
        # print(link['href'])
        auto_nuorodos.append(link['href'])

    # print(auto_nuorodos)
    # print(len(auto_nuorodos))

    if bs.find('div', {'class':'page-navigation-container'}).find('a', {'class':'next'}):
        kitas = bs.find('div', {'class':'page-navigation-container'}).find('a', {'class':'next'})
    # print(kitas['href'])
        kitas_psl = kitas['href']
        next_nuoroda = f'https://autoplius.lt{kitas_psl}'
    # print(next_nuoroda)


    for nuoroda in auto_nuorodos:
        try:
        # atsidarom kieviena nuoroda su get metodu, pasiimam reikiama info
        # info irasome i duomenu baze
            markes5 = []
            modeliai5= []
            keys5 = []
            values5 = []
            car_i5 = {}
            
            duomenys = {
            'ID':'None',
            'Marke': 'None',
            'Modelis':'None',
            'Kaina':'None',
            'Rida':'None',
            'Variklis':'None',
            'Kuro tipas':'None',
            'Pavarų dėžė':'None',
            'Varantieji ratai':'None',
            'Baterijos talpa, kWh':'None',
            'Elektra nuvažiuojamas atstumas':'None',
            '':'None',
            'Defektai':'None',
            'Spalva':'None',
            'Kėbulo tipas':'None',
            'Vairo padėtis':'None',
            'Vidutinės':'None',
            'Mieste':'None',
            'Užmiestyje':'None',
            'Pirma registracija':'None',
            'Pirmosios registracijos šalis':'None',
            'Registracijos data':'None',
            'Bendroji masė, kg':'None',
            'Nuosava masė, kg':'None',
            'Durų skaičius':'None',
            'Kėbulo numeris (VIN)':'None',
            'Ilgis':'None',
            'Aukštis':'None',
            'Tech. apžiūra iki':'None',
            'CO₂ emisija, g/km':'None',
            'Euro standartas':'None',
            'Taršos mokestis':'None',
            'SDK':'None',
            'Klimato valdymas':'None',
            'Sėdimų vietų skaičius':'None',
            'Ratlankių skersmuo':'None'
            }

            url = nuoroda
            driver.get(url)
            a = random.randint(2, 7)
            time.sleep(a)
            source = driver.page_source
            
            bs_auto = BeautifulSoup(source, 'html.parser')
            
            if bs_auto.find('div', {'class':'error'}):
                print('skelbimas neegzistuoja')
            else:
                car = bs_auto.find('ol', {'class':'breadcrumbs'}).find_all('li', {'class':'crumb'})
                marke = car[2].text.strip()
                modelis = car[3].text.strip()
                markes5.append(marke)
                modeliai5.append(modelis)
                # print(marke, modelis)
                duomenys['Marke'] = marke
                duomenys['Modelis'] = modelis
                
                

                car_price = bs_auto.find('div', {'class':'prices'})
                kaina = car_price.text.split('€')[0].strip()
                duomenys['Kaina'] = kaina
                
                if bs_auto.find('div', {'class':'col-4 announcement-id-label'}).text.split('ID:')[-1].strip():
                    idx = bs_auto.find('div', {'class':'col-4 announcement-id-label'}).text.split('ID:')[-1].strip()
                    duomenys['ID'] = idx

                car_info = bs_auto.find_all('div', {'class':'parameter-row'})
                for info in car_info[1:]:
                    # print(info)
                    # if info is not None:
                    #     # print('::NRW:::')
                    key = info.find('div', {'class':'parameter-label'}).text.strip()
                    # print(key.text.strip())
                    keys5.append(key)
                    if info.find('div', {'class':'parameter-value green-vehicle'}):
                        value = info.find('div', {'class':'parameter-value green-vehicle'}).text.strip()
                    if info.find('div', {'class':'parameter-value co2-tax approximate free'}):
                        value = info.find('div', {'class':'parameter-value co2-tax approximate free'}).text.strip()
                    else:
                        value = info.find('div', {'class':'parameter-value'}).text.strip()
                    values5.append(value)
                    # print(key, value)
                    # print(value.text.strip())
                    duomenys[key] = value

            # for k, v in zip(keys5)    
            # print(duomenys)
            data = tuple(duomenys.values())
            data1 = []
            data1.append(data)

            SDB = sqlite3.connect('Auto.db')  # jei neegzistuoja db , bus sukurta nauja db
            Cs = SDB.cursor()

            #  if not exists - tikrina ar jau sukurta DB
            sql = '''create table if not exists Autopliuslt
            (
            ID text not null,
            Marke text not null,
            Modelis text not null,
            Kaina text not null,
            Rida text not null,
            Variklis text not null,
            Kuras text not null,
            PavaruDeze text not null,
            VarantiejiRatai text not null,
            BaterijosTalpakWh text not null,
            ElektraNuvažiuojamasAtstumas text not null,
            Ikraunamas text not null,
            Defektai text not null,
            Spalva text not null,
            KebuloTipas text not null,
            Vairas text not null,
            VidutinesSanaudos text not null,
            MiesteSanaudos text not null,
            UzmiestyjeSanaudos text not null,
            PirmaRegistracija text not null,
            PirmosiosRegistracijosSalis text not null,
            RegistracijosData text not null,
            BendrojiMase text not null,
            NuosavaMase text not null,
            DuruSkaicius text not null,
            KebuloNr text not null,
            Ilgis text not null,
            Aukstis text not null,
            TechApžiuraIki text not null,
            COemisijagkm text not null,
            EuroStandartas text not null,
            TarsosMokestis text not null,
            SDK text not null,
            KlimatoValdymas text not null,
            SedimosVietos text not null,
            Ratlankiai text not null
            )
            '''
            Cs.execute(sql)

            sql_template = '''insert into Autopliuslt values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
            Cs.executemany(sql_template, data1)

            SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

            SDB.close()
        except:
            print(f'klaida su nuoroda {nuoroda}')
            pass
        

    driver.get(next_nuoroda)
    
    # su get metodu atsidarom next_nuoroda
    # nuskaitome next puslapi
    # pasiimam nuajas auto nuorodas ir nauja next nuorodas
    # cikla kartojam is naujo
    page = page +1
    print('data written to DB')
    print(f'kitas puslapis: {kitas_psl}')

driver.close()
print('Baigta!!!')

data written to DB
kitas puslapis: /skelbimai/naudoti-automobiliai?page_nr=150
klaida su nuoroda https://autoplius.lt/skelbimai/volkswagen-caddy-2-0-l-vienaturis-2023-dyzelinas-26479246.html
data written to DB
kitas puslapis: /skelbimai/naudoti-automobiliai?page_nr=151
data written to DB
kitas puslapis: /skelbimai/naudoti-automobiliai?page_nr=152
data written to DB
kitas puslapis: /skelbimai/naudoti-automobiliai?page_nr=153
data written to DB
kitas puslapis: /skelbimai/naudoti-automobiliai?page_nr=154
data written to DB
kitas puslapis: /skelbimai/naudoti-automobiliai?page_nr=155
data written to DB
kitas puslapis: /skelbimai/naudoti-automobiliai?page_nr=156
data written to DB
kitas puslapis: /skelbimai/naudoti-automobiliai?page_nr=157
data written to DB
kitas puslapis: /skelbimai/naudoti-automobiliai?page_nr=158
data written to DB
kitas puslapis: /skelbimai/naudoti-automobiliai?page_nr=159
klaida su nuoroda https://autoplius.lt/skelbimai/toyota-yaris-1-5-l-hecbekas-2024-benzinas-elektra

In [4]:
# kaip rasti markes, kiekius ir ju id

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://autoplius.lt'
driver.get(url)
time.sleep(5)

source = driver.page_source

bs = BeautifulSoup(source, 'html.parser')
driver.close()


In [ ]:
{'class': lambda x: x.startswith('ilgo pavadinimo pradžia')}

{'class': lambda x: x and x.startswith('ilgo pavadinimo pradžia')}

In [21]:
# info = bs.find_all('div', {'class':'dropdown-option js-option'})
brands = []
ids = []
counts = []

info = bs.find_all('div', class_='dropdown-option js-option', 
                               attrs={'data-badge': True, 
                                      'data-title': True, 
                                      'data-value': True})
# info = bs.find_all('div', 'data-badge')
for i in info:
    # print(':::NEXT:::')
    # print(f'{i['data-title']}, id: {i['data-value']}, kiekis: {i['data-badge']}')
    brands.append(i['data-title'])
    ids.append(i['data-value'])
    counts.append(i['data-badge'])
    
print(brands)
print(ids)
print(counts)

['-Kita-', 'Abarth', 'Acura', 'Aiways', 'Aixam', 'Alfa Romeo', 'Alpina', 'Aston Martin', 'Audi', 'Austin Rover', 'Austin-Healey', 'Bentley', 'BMW', 'Bolloré', 'Buick', 'BYD', 'Cadillac', 'Cenntro', 'Chevrolet', 'Chrysler', 'Citroen', 'Cupra', 'Dacia', 'Daewoo', 'Daihatsu', 'Dodge', 'DS Automobiles', 'Ferrari', 'Fiat', 'Ford', 'GAZ', 'GMC', 'Great Wall', 'GWM', 'Honda', 'Hudson', 'Hummer', 'Hyundai', 'Ineos', 'Infiniti', 'Isuzu', 'Iveco', 'Jaguar', 'Jeep', 'KGM', 'Kia', 'Lada', 'Lamborghini', 'Lancia', 'Land Rover', 'LDV', 'Lexus', 'Ligier', 'Lincoln', 'LuAZ', 'Lucid', 'Lynk & Co', 'MAN', 'Maserati', 'Maxus', 'Maybach', 'Mazda', 'Mercedes-Benz', 'Mercury', 'MG', 'Microcar', 'Mini', 'Mitsubishi', 'Moskvich', 'NIO', 'Nissan', 'Opel', 'Panther', 'Paxster', 'Peugeot', 'Plymouth', 'Polestar', 'Pontiac', 'Porsche', 'Renault', 'Rolls-Royce', 'Rover', 'Saab', 'Scion', 'Seat', 'Seres', 'Skoda', 'Smart', 'SsangYong', 'Subaru', 'Suzuki', 'Tatra', 'Tazzari', 'Tesla', 'Toyota', 'Triumph', 'UAZ', 'Va

In [22]:
args = ['scritas.py', 'abarth']

-Kita- 106 4
Abarth 33934 1
Acura 104 12
Aiways 33416 2
Aixam 108 35
Alfa Romeo 103 115
Alpina 112 10
Aston Martin 100 17
Audi 99 3931
Austin Rover 15642 1
Austin-Healey 27791 1
Bentley 98 51
BMW 97 6264
Bolloré 29863 1
Buick 96 38
BYD 32287 1
Cadillac 95 53
Cenntro 33744 1
Chevrolet 94 296
Chrysler 93 265
Citroen 92 977
Cupra 28897 48
Dacia 110 138
Daewoo 91 1
Daihatsu 90 17
Dodge 89 302
DS Automobiles 29763 15
Ferrari 87 4
Fiat 86 402
Ford 85 2137
GAZ 109 24
GMC 41 9
Great Wall 122 2
GWM 34099 3
Honda 84 623
Hudson 27783 1
Hummer 83 7
Hyundai 82 714
Ineos 37152 1
Infiniti 81 59
Isuzu 80 7
Iveco 125 109
Jaguar 79 255
Jeep 78 325
KGM 33844 1
Kia 77 756
Lada 76 48
Lamborghini 75 3
Lancia 74 24
Land Rover 73 527
LDV 33999 1
Lexus 72 679
Ligier 18723 23
Lincoln 71 21
LuAZ 119 3
Lucid 33181 1
Lynk & Co 33720 1
MAN 30847 30
Maserati 69 69
Maxus 35741 1
Maybach 111 1
Mazda 68 564
Mercedes-Benz 67 3552
Mercury 66 2
MG 65 10
Microcar 16127 11
Mini 64 338
Mitsubishi 63 345
Moskvich 116 14
NIO 3